<a href="https://colab.research.google.com/github/muziejus/coms-w4995-applied-machine-learning-project/blob/main/notebooks/combine_sentiment_and_financial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


## Set constants

In [12]:
root_data_url = "https://github.com/muziejus/coms-w4995-applied-machine-learning-project/raw/refs/heads/main/data"
financial_data_url = "financial_data"
sentiment_article_number = 75000
sentiment_data_url = f"sentiment_data/analyzed_{sentiment_article_number}"
sentiment_file_name_tail = f"_x_{sentiment_article_number}.parquet"

companies = ["dltr", "lulu", "ulta", "wba", "wmt"]

In [ ]:
fin_df = pd.read_csv(f"{root_data_url}/{financial_data_url}/merged_data.csv")

In [9]:
fin_df.head()

,Date,Open_lulu,High_lulu,Low_lulu,Close_lulu,Volume_lulu,Dividends_lulu,Stock Splits_lulu,Open_wmt,High_wmt,...,Dividends_dltr,Stock Splits_dltr,CPIAUCSL,PCE,PPIACO,ECIALLCIV,GDPDEF,UNRATE,MCUMFN,SP500
0,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,252.561,14084.6,199.1,136.1,103.328,4.0,78.2051,NaN
1,2019-01-02,118.889999,124.589996,118.279999,123.349998,2085800.0,0.0,0.0,27.838942,28.449552,...,0.0,0.0,252.561,14084.6,199.1,136.1,103.328,4.0,78.2051,2510.03
2,2019-01-03,121.849998,126.120003,120.320000,124.360001,2896800.0,0.0,0.0,28.315889,28.771568,...,0.0,0.0,252.561,14084.6,199.1,136.1,103.328,4.0,78.2051,2447.89
3,2019-01-04,125.849998,129.570007,125.309998,128.550003,2184600.0,0.0,0.0,28.315886,28.452589,...,0.0,0.0,252.561,14084.6,199.1,136.1,103.328,4.0,78.2051,2531.94
4,2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,252.561,14084.6,199.1,136.1,103.328,4.0,78.2051,2531.94


## Create aggregated sentiment data

In [19]:
sentiment_df.head()

,index,goid,date,tokens,corpus,daily_article_count,daily_token_sum,text_sentiment,text_error,text_input_tokens
0,0,3094664002,2024-08-20,1226,lululemon,3263,2795077,0.457104,336.445007,1758.0
1,1,2267936832,2019-08-02,2057,lululemon,21,59999,0.095200,18.105124,2792.0
2,2,2645172457,2022-03-30,710,lululemon,1899,1719875,-0.074508,21.908319,1021.0
3,3,3070094485,2024-06-21,8152,lululemon,58,261166,0.045668,41.133335,10651.0
4,4,2424011426,2020-07-16,1090,lululemon,72,110246,-0.035261,208.958359,1850.0


In [29]:
def agg_weighted_avg(row, value_column, weight_column):
    return (row[value_column] * row[weight_column]).sum() / row[weight_column].sum()

for company in ["lulu"]: # companies
  sent_df = pd.read_parquet(f"{root_data_url}/{sentiment_data_url}/{company}{sentiment_file_name_tail}")
  agg_sent_df = sent_df.groupby("date").agg(
      analyzed_bert_tokens = ("text_input_tokens", "sum"),
      weighted_sentiment=("text_input_tokens", lambda row: weighted_avg(sent_df.loc[row.index], "text_sentiment", "text_input_tokens")),
      weighted_error=("text_input_tokens", lambda row: weighted_avg(sent_df.loc[row.index], "text_error", "text_input_tokens")),
      analyzed_naive_tokens = ("tokens", "sum"),
      daily_naive_token_sum = ("daily_token_sum", "first"),
      analyzed_article_count = ("index", "count"),
      daily_article_count = ("daily_article_count", "first"),
  )

In [31]:
agg_sent_df.head()

,analyzed_bert_tokens,weighted_sentiment,weighted_error,analyzed_naive_tokens,daily_naive_token_sum,analyzed_article_count,daily_article_count
date,,,,,,,
2019-01-01,78837.0,0.195344,41.043721,57841,60358,12,15
2019-01-02,74422.0,0.180192,158.752229,39626,42472,24,27
2019-01-03,115014.0,0.126784,156.380061,37031,37530,15,16
2019-01-04,118195.0,0.074154,126.906331,41872,45767,14,17
2019-01-07,109520.0,0.069071,114.400445,35020,36574,16,17
